## 절차

1. 연구목표 설정
2. 데이터 획득(수집)
  - https://play.google.com/store/apps/details?id=com.nhn.android.webtoon&showAllReviews=true
  - https://play.google.com/store/apps/details?id=net.daum.android.webtoon&showAllReviews=true
  - selenium 이용
  - 페이지로드되면,  END 를 작동시켜 밑 밑으로 내린다 -> 10번(임의설정), 3초간대기
  - 리뷰를 수집해 온다
   > 이름   
   > 별점  
   > 등록일  
   > 내용  
   > 도움이됬음  
  
3. 데이터 준비(정제,변환,통합)
   > 정규식으로 필터링 (욕설, 기타 표현에대한 정책)
   > 파생변수 -> 형태소, 점수 지표를 이용한 계산, 사전에 없는 표현에 대한 유사어검사(확장성) 등 
4. 데이터 탐색(시각화,통찰과정)
   > 워드클라우드, 바차트, 히스토그램, 산포도
5. 생략
6. 시스템통합(사전의 재구축(선순환구조) 및 반영, 서비스)

In [24]:
from selenium import webdriver as wd
import urllib

In [25]:
target_urls = [
    'https://play.google.com/store/apps/details?id=com.nhn.android.webtoon&showAllReviews=true',
    'https://play.google.com/store/apps/details?id=net.daum.android.webtoon&showAllReviews=true'    
]

In [26]:
driver = wd.Chrome('../tool/chromedriver_75.0.3770.140.exe')

In [27]:
from selenium.webdriver.common.keys import Keys
def getData(app_name):
    # 타겟 사이트별 데이터 수집 작업
    # 스크롤바를 밑으로 내린다 (10번)
    for n in range(10):
        # selenium.webdriver.common.keys.Keys
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        # ajax 통해서 리뷰가 로드될것이다
        time.sleep(3)
        #break
    
    reviews = driver.find_elements_by_css_selector('.W4P4ne>div:nth-child(2)>div>div')
    # 전체 리뷰수
    print( len(reviews) )
    tmp = list()
    try:
        for div in reviews:
            # 리뷰를 하나씩 꺼내서
            name    = div.find_element_by_class_name('X43Kjb')            
            star    = div.find_element_by_css_selector('.pf5lIe>div')
            regi    = div.find_element_by_class_name('p2TkOb')
            content = div.find_element_by_css_selector('.UD7Dzf>span')
            good    = div.find_element_by_class_name('jUL89d.y92BAb')
            dic     = {
                'name':name.text,
                'star':star.get_attribute('aria-label'),
                'regi':regi.text,
                'content':content.text,
                'good':good.text,
                'app_name':app_name
            }
            tmp.append( dic )
    except Exception as e:
        print('실제로는 데이터가 없으나 있는것으로 처리되서 커트')
    return tmp
        
    
    
    # 수집하고자 하는 내용들을 완벽하게 순서대로 세트를 유지하고, class명이 모두 동일
    # 여기서는 개별적으로 구해서 순서대로 처리    
    # 후기가 있어서 수치가 틀어진다 -> 보정이 필요
    # 이름   
    #names    = driver.find_elements_by_class_name('X43Kjb')
    # 별점 
    #stars    = driver.find_elements_by_css_selector('.pf5lIe>div')
    # 등록일
    #regis    = driver.find_elements_by_class_name('p2TkOb')
    # 내용  
    #contents = driver.find_elements_by_css_selector('.UD7Dzf>span')
    # 도움이됬음 
    #goods    = driver.find_elements_by_class_name('jUL89d.y92BAb')
    #print( len(names),len(stars),len(regis),len(contents),len(goods)  )    

In [28]:
import time
outputs   = list()
app_names = ['네이버웹툰','다음웹툰']
for idx, target_url in enumerate(target_urls):    
    driver.get( target_url )
    outputs.extend( getData( app_names[idx] ) )
    time.sleep(4)
    #break

201
실제로는 데이터가 없으나 있는것으로 처리되서 커트
120


In [29]:
len(outputs)

320

In [30]:
outputs[-1:]

[{'name': '강지우',
  'star': '별표 5개 만점에 1개를 받았습니다.',
  'regi': '2019년 8월 7일',
  'content': '로그인이 안돼요 카카오계정으로 로그인하라는데 카카오계정으로 로그인하기 버튼도 없고 뭘 하라는건지',
  'good': '',
  'app_name': '다음웹툰'}]

In [31]:
import pandas as pd
import numpy as np

In [32]:
df = pd.DataFrame.from_dict( outputs )
df.head(2)

,app_name,content,good,name,regi,star
0,네이버웹툰,쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을...,1,솜담,2019년 8월 17일,별표 5개 만점에 1개를 받았습니다.
1,네이버웹툰,저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를...,2,왈왈,2019년 8월 20일,별표 5개 만점에 1개를 받았습니다.


In [33]:
# star에서 점수만 추출
sample = '별표 5개 만점에 3개를 받았습니다.'
import re

In [34]:
p = re.compile( '\d+' )

In [35]:
p.findall( sample )[1]

'3'

In [36]:
# star 값들을 실제 점수로만 변경한다
df['star'] = df['star'].apply( lambda x: p.findall( x )[1]  )

In [37]:
df.dtypes

app_name    object
content     object
good        object
name        object
regi        object
star        object
dtype: object

In [38]:
# regi에서 날자를 YYYY-mm-dd 형태를 날짜 포멧팅으로 처리
# step 1. 년도, 월, 일 값들을 일단 뽑아낸다
# "2019년 8월 16일" => (\d+).+(\d+).+(\d+).
p        = re.compile( '(\d+)\w\s(\d+)\w\s(\d+)\w' )
sample   = "2019년 8월 16일" 
#date_raw = p.search( sample ).groups()
# 2019-8-16
p.sub( '\g<1>-\g<2>-\g<3>', sample )

'2019-8-16'

In [39]:
import datetime

In [40]:
datetime.datetime.strptime( '2019-8-16', '%Y-%m-%d' ).date()

datetime.date(2019, 8, 16)

In [41]:
# df에 regi 항목에 데이터를 datetime.date(2019, 8, 16) 타입으로 치환하시오
df['regi'] = df['regi'].apply( lambda x: 
                  datetime.datetime.strptime( p.sub( '\g<1>-\g<2>-\g<3>', x ) 
                                              , '%Y-%m-%d' ).date()  )

In [42]:
df.head(2)

,app_name,content,good,name,regi,star
0,네이버웹툰,쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을...,1,솜담,2019-08-17,1
1,네이버웹툰,저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를...,2,왈왈,2019-08-20,1


In [43]:
# content에서 형태소 분석을 위한 영어 제거 혹은 대체?
df['content']

0      쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을...
1      저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를...
2      "개를 낳았다"를 즐겨보고 있는데 엄청 재밌드라고요 그런데 개를 낳았다를보고 다른걸...
3      한 두 번도 아니고 와이파이, 데이터 둘 다 네이버웹툰만 켜면 제일을 왜 못합니까?...
4      아니;;장난하시냐고요--여러분 이거요기요 첫주문웬만하면하지마세요.쿠키130개?절대안...
5      자기의 취향에 걸맞는 것도 찾기가 쉽고 또한 어느것이 도 인기가 도 많은지 바로바로...
6      아니 왜 무료쿠키 안줘요? 앱깔고 회원가입하면 쿠키 9개 준다길래 앱깔고 회원가입도...
7      안녕하세요. 제가 할거가 없어서 찾고 있었는데 딱 이게 발견 했습니다. 그래서 한번...
8      요즘 자꾸 모자이크칸이 많이뜨고 이미지가 안나오네요. 크롬으로 네이버직좁 들어가서 ...
9      진짜 진짜 재밌게 보고 있어요 !! 💗 근데 와이파이 없이 볼수는 없나요 ? ㅜ 고...
10     쿠키 첫주문 이벤트 아직까지 안주더라고요.. 그리고 어제부터 갑자기 다른 웹툰보려고...
11     저기요 돼지만화는 언제 자르실건지.....? 그거 보면 진심 화나요..... 조선팔...
12     네이버 웹툰은 컷마다 볼 수 있는 댓글과 보기편한 점이 있어서 좋지만 다른 새로운 ...
13     결제는 네이버 어플에서 바로하게하고 미리볼려면 다른 어플(네이버웹툰어플)을 깔아야한...
14     제가 계정을 만들려고 아이디 비밀번호 만들고 전화번호도 쳤고 인증번호가 안오는것 같...
15     저만 그런건지는 모르겠는데 웹툰 밑으로 스크롤해서 보다가 중간에 만화 장면이 나타나...
16     저 의견사항에서 얘기 했는데 답장이 하도 안와서 여기서 말합니다 제 오해 일 수 있...
17     물론 아주 편리한 앱입니다 하지만 하나하나 

In [44]:
# 백업
df.to_csv( './data/app_review_mid.xls', encoding='utf-8'  )

In [45]:
# 명사, 형용사 기준으로 하고, 동사는 확장성/가능성을 두고 체크
# 형태소 분석을 하기 전에 사전에 포함되어 있지 않는 유형의 문자들을 제거
# 1. 영어, 숫자 제거 <-> 한글과 특수기호, 화이트스페이스, 추출
tmp = df['content'].copy()

In [46]:
for doc in tmp:
    print( doc )

쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을 얼마나 많이했는데 방금 알바가입 머시했거든요? 10개 빨리주세요;; 진짜 나말고도 많더만 이정도면 네이버 이거 고객기만이지 장난하나;;;; 그리고 작가님들 다 좋거든요? 근데 왜 자꾸 하위권짜르세요 장난하나 작가님들 작품이 장난이에요? 이상한 만화는 안 짜르고 스토리 탄탄에 작화 좋은데 하위권 다 짤라;;;
저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를 확인하라고 하고;; 삭제 하고 다시 로그인 할려고 하면 이용약관에 동의 하라고 하고 진짜 화나서 욕이 나올려고 하네요;;
"개를 낳았다"를 즐겨보고 있는데 엄청 재밌드라고요 그런데 개를 낳았다를보고 다른걸 한번도 본적이 없었어요 그런데 개를 낳았다 끝까지 봐서 다음화 나올때까지 다른걸 봤는데 재밌어서지금은 여러가지를 보고 있어요 전 제일 좋아하는 3가지가 있어요 1위 개를 낳았다 2위 유미의 세포들 3위 날 가져요 넘나 재밌어요ㅠㅠ 전 이 앱을 깔고 후회한적이 없어요ㅠ 전 그래서 별 5개를 드립니다!!
한 두 번도 아니고 와이파이, 데이터 둘 다 네이버웹툰만 켜면 제일을 왜 못합니까? 데이터 충분하고 카톡 잘됩니다. 지금 별점도 데이터로 남기는데 웹툰 이미지 파일만 로딩이 안됩니다. 캐시 삭제까지 해봤습니다^^;
아니;;장난하시냐고요--여러분 이거요기요 첫주문웬만하면하지마세요.쿠키130개?절대안줌 내가장담함.이용문의에도 글을몇번이나올렸는데 안줌 이글을읽는 네이버관계자분들 제발좀 쿠키좀주시죠; 이거지금 10일이상넘음; 이정도면 네이버진짜일안하는수준
자기의 취향에 걸맞는 것도 찾기가 쉽고 또한 어느것이 도 인기가 도 많은지 바로바로 알수 가 있어서 좋아용. 그런데 쿠키는 좀 몇개를 좀 없앴으면 좋겠어요. 그리고 쓰는 방법도 잘 모르겠고 이무튼 이런저런것들을 좀 수정해주세요.
아니 왜 무료쿠키 안줘요? 앱깔고 회원가입하면 쿠키 9개 준다길래 앱깔고 회원가입도 성공했는데 주지도 않고 다시 이 앱 들

In [47]:
p = re.compile( '[a-zA-Z0-9]+' ) 

In [48]:
m = p.sub( ' ', tmp[0] )

In [49]:
m

'쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을 얼마나 많이했는데 방금 알바가입 머시했거든요?  개 빨리주세요;; 진짜 나말고도 많더만 이정도면 네이버 이거 고객기만이지 장난하나;;;; 그리고 작가님들 다 좋거든요? 근데 왜 자꾸 하위권짜르세요 장난하나 작가님들 작품이 장난이에요? 이상한 만화는 안 짜르고 스토리 탄탄에 작화 좋은데 하위권 다 짤라;;;'

In [50]:
# 전체적용
df['content'] = df['content'].apply( lambda x: p.sub( ' ', x ) )

In [51]:
# 컬럼 하나 추가
# star 컬럼을 참조하여 
# 긍정 : 별점기준 4 or 5       star > 3
# 부정 : 별점기준 1 or 2 or 3  star <= 3
# setiment : 'pos' or 'neg'

In [52]:
df['star'][:5]

0    1
1    1
2    5
3    1
4    1
Name: star, dtype: object

In [53]:
df['star'].dtype

dtype('O')

In [54]:
df['star'] = df['star'].astype(int)

In [55]:
df['star'].dtype

dtype('int32')

In [56]:
df.dtypes

app_name    object
content     object
good        object
name        object
regi        object
star         int32
dtype: object

In [57]:
def checkPosNeg( x ):
    if x>3:return 'pos'
    return 'neg'

In [66]:
#vx>3 and 'pos' or 'neg'

In [61]:
df['star'].apply( lambda x: x>3 and 'pos' or 'neg' )[:2]

0    neg
1    neg
Name: star, dtype: object

In [62]:
df['setiment'] = df['star'].apply( checkPosNeg )

In [63]:
df['setiment'][:2]

0    neg
1    neg
Name: setiment, dtype: object

In [74]:
# 마스크 인자로 조건이 참이녀석 NaN  or  대체
df['star'].mask( df['star'] > 3, 'pos' )[:5]

0      1
1      1
2    pos
3      1
4      1
Name: star, dtype: object

In [76]:
df.head(2)

,app_name,content,good,name,regi,star,setiment
0,네이버웹툰,쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을...,1,솜담,2019-08-17,1,neg
1,네이버웹툰,저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를...,2,왈왈,2019-08-20,1,neg


In [77]:
# 조건이 거짓인 데이터를 찾아서 Nan or  대체
df['star'].where( df['star'] > 3, 'neg' )[:5]

0    neg
1    neg
2      5
3    neg
4    neg
Name: star, dtype: object

In [78]:
df.head(2)

,app_name,content,good,name,regi,star,setiment
0,네이버웹툰,쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을...,1,솜담,2019-08-17,1,neg
1,네이버웹툰,저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를...,2,왈왈,2019-08-20,1,neg


In [ ]:
# 트위터 형태소 분석기를 이용하여 품사를 포함한 분해 함수를 이용하여 
# 새로운 컬럼 classifier라는 항목에 리뷰를 형태소로 분해한 내용으로 채운다.

In [79]:
from konlpy.tag import Okt
pos_tagger = Okt()

In [80]:
tmp0 = pos_tagger.pos( df['content'][0] )
len(tmp0)

90

In [81]:
# 형태소 분석시 정규화및 어근화 처리를 수행했다 <- 선택은 옵션
tmp1 = pos_tagger.pos( df['content'][0], norm=True, stem=True )
len( tmp1 )

90

In [82]:
tmp1[:4]

[('쿠키', 'Noun'), ('왜', 'Noun'), ('알다', 'Verb'), ('몇번', 'Noun')]

In [84]:
# 대상을 좁힌다 => 명사 : Noun, 형용사 : Adjective, 동사(예비):Verb
# 위의것들만 남기고 제거
tags = [ 'Noun', 'Adjective', 'Verb' ]
# for word, tag in tmp1:
#     if tag in tags:
#         print( word, tag )
#[ ( word, tag ) for word, tag in tmp1 if tag in tags ]

[ tag for tag in tmp1 if tag[1] in tags ]

[('쿠키', 'Noun'),
 ('왜', 'Noun'),
 ('알다', 'Verb'),
 ('몇번', 'Noun'),
 ('진짜', 'Noun'),
 ('무료', 'Noun'),
 ('쿠키', 'Noun'),
 ('이르다', 'Verb'),
 ('알다', 'Verb'),
 ('번인', 'Noun'),
 ('내', 'Noun'),
 ('가입', 'Noun'),
 ('얼마나', 'Noun'),
 ('하다', 'Verb'),
 ('방금', 'Noun'),
 ('알바', 'Noun'),
 ('가입', 'Noun'),
 ('머시', 'Noun'),
 ('하다', 'Verb'),
 ('개', 'Noun'),
 ('줄다', 'Verb'),
 ('진짜', 'Noun'),
 ('나', 'Noun'),
 ('많다', 'Adjective'),
 ('정도', 'Noun'),
 ('네이버', 'Noun'),
 ('거', 'Noun'),
 ('고객', 'Noun'),
 ('기만', 'Noun'),
 ('장난', 'Noun'),
 ('하나', 'Noun'),
 ('작가', 'Noun'),
 ('들다', 'Verb'),
 ('좋다', 'Adjective'),
 ('왜', 'Noun'),
 ('자꾸', 'Noun'),
 ('하위', 'Noun'),
 ('짜르', 'Noun'),
 ('세', 'Noun'),
 ('장난', 'Noun'),
 ('하나', 'Noun'),
 ('작가', 'Noun'),
 ('들다', 'Verb'),
 ('작품', 'Noun'),
 ('장난', 'Noun'),
 ('이상하다', 'Adjective'),
 ('만화', 'Noun'),
 ('안', 'Noun'),
 ('짜르', 'Noun'),
 ('스토리', 'Noun'),
 ('탄탄', 'Noun'),
 ('작화', 'Noun'),
 ('좋다', 'Adjective'),
 ('하위', 'Noun'),
 ('짤', 'Noun')]

In [88]:
[ tag 
  for tag in pos_tagger.pos( df['content'][0], norm=True, stem=True ) 
  if tag[1] in tags ][:2]

[('쿠키', 'Noun'), ('왜', 'Noun')]

In [91]:
df['content'].apply( lambda x: [ tag 
  for tag in pos_tagger.pos( x, norm=True, stem=True ) 
  if tag[1] in tags ] )

0      [(쿠키, Noun), (왜, Noun), (알다, Verb), (몇번, Noun)...
1      [(저기, Noun), (갑, Noun), (작다, Adjective), (자다, ...
2      [(개, Noun), (낳다, Verb), (를, Noun), (즐기다, Verb)...
3      [(하다, Verb), (두, Noun), (번, Noun), (아니다, Adjec...
4      [(아니다, Adjective), (장난, Noun), (하다, Verb), (고요...
5      [(자기, Noun), (취향, Noun), (걸, Noun), (맞다, Verb)...
6      [(아니다, Adjective), (왜, Noun), (무료, Noun), (쿠키,...
7      [(안녕하다, Adjective), (제, Noun), (하다, Verb), (없다...
8      [(요즘, Noun), (자꾸, Noun), (모자이크, Noun), (칸, Nou...
9      [(진짜, Noun), (진짜, Noun), (재밌다, Adjective), (보고...
10     [(쿠키, Noun), (주문, Noun), (이벤트, Noun), (알다, Ver...
11     [(저기, Noun), (돼지, Noun), (만화, Noun), (언제, Noun...
12     [(네이버, Noun), (웹툰, Noun), (컷, Noun), (볼, Noun)...
13     [(결제, Noun), (네이버, Noun), (어플, Noun), (바로, Nou...
14     [(제, Noun), (계정, Noun), (만들다, Verb), (아이디, Nou...
15     [(그렇다, Adjective), (모르다, Verb), (웹툰, Noun), (밑...
16     [(저, Noun), (의견, Noun), (사항, Noun), (얘기, Noun)...
17     [(아주, Noun), (편리하다, Adje

In [92]:
df['classifier']=df['content'].apply( lambda x: [ tag 
  for tag in pos_tagger.pos( x, norm=True, stem=True ) 
  if tag[1] in tags ] )

In [93]:
df['classifier'][:2]

0    [(쿠키, Noun), (왜, Noun), (알다, Verb), (몇번, Noun)...
1    [(저기, Noun), (갑, Noun), (작다, Adjective), (자다, ...
Name: classifier, dtype: object

In [94]:
df.head(2)

,app_name,content,good,name,regi,star,setiment,classifier
0,네이버웹툰,쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을...,1,솜담,2019-08-17,1,neg,"[(쿠키, Noun), (왜, Noun), (알다, Verb), (몇번, Noun)..."
1,네이버웹툰,저기요... 갑작히 잘 되던게 왜 안 됩니까? 와이파이 빵빵한데 네트워크 연결상태를...,2,왈왈,2019-08-20,1,neg,"[(저기, Noun), (갑, Noun), (작다, Adjective), (자다, ..."


In [95]:
import json

In [101]:
# 감정사전, 대략 7만여개의 데이터에 긍정/부정지수값이 세팅된 사전
with open('./data/SentiWord_info.json', encoding='utf-8') as f:
    score_dic = json.load( f )

In [102]:
score_dic

[{'word': '(-;', 'word_root': '(', 'polarity': '1'},
 {'word': '(;_;)', 'word_root': '(;_;)', 'polarity': '-1'},
 {'word': '(^^)', 'word_root': '(^^)', 'polarity': '1'},
 {'word': '(^-^)', 'word_root': '(^-^)', 'polarity': '1'},
 {'word': '(^^*', 'word_root': '(', 'polarity': '1'},
 {'word': '(^_^)', 'word_root': '(^_^)', 'polarity': '1'},
 {'word': '(^_^;', 'word_root': '(', 'polarity': '-1'},
 {'word': '(^o^)', 'word_root': '(^o^)', 'polarity': '1'},
 {'word': '(-_-)', 'word_root': '(-_-)', 'polarity': '-1'},
 {'word': '(T_T)', 'word_root': '(T_T)', 'polarity': '-1'},
 {'word': '(ㅡㅡ)', 'word_root': '(', 'polarity': '-1'},
 {'word': ')-:', 'word_root': ')', 'polarity': '-1'},
 {'word': '*^^*', 'word_root': '*', 'polarity': '1'},
 {'word': '/^o^\\', 'word_root': '/', 'polarity': '1'},
 {'word': ':(', 'word_root': ':', 'polarity': '1'},
 {'word': ":'-(", 'word_root': ':', 'polarity': '1'},
 {'word': ':-(', 'word_root': ':', 'polarity': '1'},
 {'word': ':)', 'word_root': ':', 'polarity':

In [ ]:
# 리뷰에 대한 감정사전으로 평가한 점수표를 새로운 컬럼 score로 대입한다.
# 비교대상 word를 비교하여 평가를 수행, 점수는 누적합
# 비교대상이 없는 경우 일단 누락 및 따로 모아서 [ (), (), ] noncheck으로 추가

In [105]:
# 샘플 1개에 대해서 점수를 계산해본다.
df['classifier'][0][:2]

[('쿠키', 'Noun'), ('왜', 'Noun')]

In [111]:
polarity_=0
for word, tag in df['classifier'][0]:
    for dic in score_dic:
        if dic['word']==word:
                print( word, dic['polarity'])
                polarity_+= int(dic['polarity'])

polarity_, df['content'][0], df['star'][0]

많다 2
좋다 2
이상하다 -1
좋다 2


(5,
 '쿠키 왜 안주세요 몇번째야 진짜 무료쿠키~ 이러면서 안주는게 한두번인가 내가 가입을 얼마나 많이했는데 방금 알바가입 머시했거든요?  개 빨리주세요;; 진짜 나말고도 많더만 이정도면 네이버 이거 고객기만이지 장난하나;;;; 그리고 작가님들 다 좋거든요? 근데 왜 자꾸 하위권짜르세요 장난하나 작가님들 작품이 장난이에요? 이상한 만화는 안 짜르고 스토리 탄탄에 작화 좋은데 하위권 다 짤라;;;',
 1)

- 형태소를 어떻게 처리했느냐, 매칭이 얼마나 되느냐,
- 사전의 표현은 얼마나 풍성하느냐에 따라 긍정과 부정이 계속 바뀐다.
- ==> 사람이 인위적으로 계속해서 사전을 추가하는 것은 불가능하다
- ==> 매칭이 안된 단어들은 유사도를 계산하여(백터화 거리계산법) 가장 근접한 표현으로 찾아서
- 점수화 하면 많이 보완 될 것이고, 이것이 사전에 반영되면 계산없이 바로 찾을 수도 있을 것이다.
- **궁극적으로 맥락이라는 것이 더 중요하므로, 머신러닝이나 딥러닝**으로 처리하는게 보다 효율적이다.
- 챗복이라는 서비스, 번역 등등 머신러닝, 딥러닝기반구성



In [119]:
def scoreCalculator(x):
    polarity_=0
    for word, tag in x:
        for dic in score_dic:
            if dic['word']==word:
                    # print( word, dic['polarity'])
                    polarity_+= int(dic['polarity'])
    return polarity_

In [121]:
df['score'] = df['classifier'].apply( scoreCalculator )

In [122]:
df.loc[ :,['star','score']]

,star,score
0,1,5
1,1,-4
2,5,0
3,1,-1
4,1,0
5,5,5
6,2,1
7,3,0
8,1,-1
9,4,4
